# Conversion of the Latest Dataframe to Parquet

We need to store our dataset in a warehouse so we use parquet

In [9]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
# Load the Pandas DataFrame
df = pd.read_csv('dialogues.csv', sep='\t', encoding='utf-8')

In [10]:
df.to_parquet('./data/parquet/dialogues.parquet')